In [1]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import math
from tsmom_model import *
import numpy as np
from datetime import datetime
import statsmodels.api as sm

/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
import pylab
sns.set(font_scale=3)
pylab.rcParams['figure.figsize'] = (24, 18)

In [3]:
funding_liquidity, market_liquidity = liquidity_observables()

tsmom_model.py:422: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  funding_liquidity['TED Spread']=ted.resample(rule='m',how='last')/100.
tsmom_model.py:423: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  funding_liquidity['LIBOR term repo']=libor_term_repo.resample(rule='m',how='last')/100.
tsmom_model.py:427: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  PS=pd.read_csv('PS.csv',index_col=0,parse_dates=True).resample(rule='m',how='last')
tsmom_model.py:428: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  us10y_zero=quandl.get('FED/SVENY',authtoken=token).SVENY10.resample(rule='d',how='last').dropna()
tsmom_model.py:429: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  yield10=quandl.get('USTREASURY/YIELD',authtoken=token)['10 YR'].resample(rule='d',how='last').dropna()
tsmom_

In [4]:
factors=get_all_factors(3)

In [5]:
results_df=pd.DataFrame()
for reg_fact in funding_liquidity.columns:
    results_param=[]
    results_t=[]
    for s in factors.columns:
        ind2=funding_liquidity.diff()[reg_fact].dropna().index
        ind=factors[s].loc[ind2].dropna().index
        # Please make the fit robust 
        res=sm.OLS(factors[s].loc[ind],sm.add_constant(funding_liquidity.diff()[reg_fact]).loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
        results_param.append(res.params[reg_fact])
        results_t.append(res.tvalues[reg_fact])
    results_df[reg_fact]=pd.Series(results_param)
    results_df[reg_fact+'Tstats']=pd.Series(results_t)
#results_df=results_df.set_index(factors.columns)

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:7: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  import sys


In [6]:
results_df.round(2).T

,0,1,2,3
TED Spread,-0.88,-0.83,-0.60,-1.03
TED SpreadTstats,-1.89,-0.75,-0.97,-1.93
LIBOR term repo,0.41,0.48,0.49,-0.11
LIBOR term repoTstats,2.11,1.51,2.48,-0.94


In [7]:
#results_df=pd.DataFrame()
for reg_fact in market_liquidity.columns:
    results_param=[]
    results_t=[]
    for s in factors.columns:
        ind2=market_liquidity.diff()[reg_fact].dropna().index
        ind=factors[s].loc[ind2].dropna().index
        # Please make the fit robust 
        res=sm.OLS(factors[s].loc[ind],sm.add_constant(market_liquidity.diff()[reg_fact]).loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
        results_param.append(res.params[reg_fact])
        results_t.append(res.tvalues[reg_fact])
    results_df[reg_fact]=pd.Series(results_param)
    results_df[reg_fact+'Tstats']=pd.Series(results_t)
results_df=results_df.set_index(factors.columns)

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:7: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  import sys


In [8]:
results_df

,TED Spread,TED SpreadTstats,LIBOR term repo,LIBOR term repoTstats,PS,PSTstats,On versus off the run Treasuries,On versus off the run TreasuriesTstats
Amihud Monthly,-0.879314,-1.892913,0.408362,2.114939,0.002136,0.102938,-0.017477,-0.578873
FHT Monthly,-0.826584,-0.746422,0.475181,1.505960,0.009716,0.493023,-0.028403,-1.005903
Amihud Annual,-0.595745,-0.966307,0.486469,2.477186,0.021388,0.965974,0.031511,0.906252
FHT Annual,-1.027071,-1.929428,-0.107557,-0.940322,0.000705,0.039383,-0.060242,-2.155171


In [9]:
print results_df.T[[u'Amihud Annual', u'Amihud Monthly', u'FHT Annual', u'FHT Monthly']].round(2).to_latex()

\begin{tabular}{lrrrr}
\toprule
{} &  Amihud Annual &  Amihud Monthly &  FHT Annual &  FHT Monthly \\
\midrule
TED Spread                             &          -0.60 &           -0.88 &       -1.03 &        -0.83 \\
TED SpreadTstats                       &          -0.97 &           -1.89 &       -1.93 &        -0.75 \\
LIBOR term repo                        &           0.49 &            0.41 &       -0.11 &         0.48 \\
LIBOR term repoTstats                  &           2.48 &            2.11 &       -0.94 &         1.51 \\
PS                                     &           0.02 &            0.00 &        0.00 &         0.01 \\
PSTstats                               &           0.97 &            0.10 &        0.04 &         0.49 \\
On versus off the run Treasuries       &           0.03 &           -0.02 &       -0.06 &        -0.03 \\
On versus off the run TreasuriesTstats &           0.91 &           -0.58 &       -2.16 &        -1.01 \\
\bottomrule
\end{tabular}



In [10]:
# Doing each leg of the observable direclty
#results_df=pd.DataFrame()
for reg_fact in market_liquidity.columns:
    results_param=[]
    results_t=[]
    for s in factors.columns:
        ind2=market_liquidity.diff()[reg_fact].dropna().index
        ind=factors[s].loc[ind2].dropna().index
        # Please make the fit robust 
        res=sm.OLS(factors[s].loc[ind],sm.add_constant(market_liquidity.diff()[reg_fact]).loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
        results_param.append(res.params[reg_fact])
        results_t.append(res.tvalues[reg_fact])
    results_df[reg_fact]=pd.Series(results_param)
    results_df[reg_fact+'Tstats']=pd.Series(results_t)
#results_df=results_df.set_index(factors.columns)#

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:8: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  


In [11]:
ind2=market_liquidity.diff().dropna().index
ind=factors[s].loc[ind2].dropna().index
# Please make the fit robust 
res=sm.OLS(factors[s].loc[ind],sm.add_constant(market_liquidity.diff()).loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
#results_param.append(res.params[reg_fact])
#results_t.append(res.tvalues[reg_fact])

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  


In [12]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             FHT Annual   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     2.343
Date:                Thu, 06 Sep 2018   Prob (F-statistic):             0.0988
Time:                        14:48:55   Log-Likelihood:                 458.71
No. Observations:                 192   AIC:                            -911.4
Df Residuals:                     189   BIC:                            -901.7
Df Model:                           2                                         
Covariance Type:                  HAC                                         
====================================================================================================
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                                0.0009      0.002      0.562      0.574      -0.002       0.004
PS                                   0.0019      0.018      0.107      0.915      -0.033       0.037
On versus off the run Treasuries    -0.0604      0.028     -2.162      0.031      -0.115      -0.006
==============================================================================
Omnibus:                        1.694   Durbin-Watson:                   2.093
Prob(Omnibus):                  0.429   Jarque-Bera (JB):                1.688
Skew:                           0.224   Prob(JB):                        0.430
Kurtosis:                       2.902   Cond. No.                         19.1
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 1 lags and without small sample correction
"""

# Showing the results of all factors and betas

In [13]:
all_macro=market_liquidity.join(funding_liquidity)

for s in factors.columns:

    result_list=[]
    t_list=[]

    for reg_fact in funding_liquidity.columns:
        ind2=funding_liquidity[reg_fact].dropna().index
        ind=factors[s].loc[ind2].dropna().index
        # Please make the fit robust 
        res=sm.OLS(factors[s].loc[ind],sm.add_constant(funding_liquidity[reg_fact]).loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
        result_list.append(res.params)
        t_list.append(res.tvalues)

    # All funding funding macro data
    ind2=funding_liquidity.dropna().index
    ind=factors[s].loc[ind2].dropna().index
    # Please make the fit robust 
    res=sm.OLS(factors[s].loc[ind],sm.add_constant(funding_liquidity).loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    result_list.append(res.params)
    t_list.append(res.tvalues)

    for reg_fact in market_liquidity.columns:
        ind2=market_liquidity[reg_fact].dropna().index
        ind=factors[s].loc[ind2].dropna().index
        # Please make the fit robust 
        res=sm.OLS(factors[s].loc[ind],sm.add_constant(market_liquidity[reg_fact]).loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
        result_list.append(res.params)
        t_list.append(res.tvalues)

    # All market funding macro data
    ind2=market_liquidity.dropna().index
    ind=factors[s].loc[ind2].dropna().index
    # Please make the fit robust 
    res=sm.OLS(factors[s].loc[ind],sm.add_constant(market_liquidity).loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    result_list.append(res.params)
    t_list.append(res.tvalues)

    # All Macro
    ind2=all_macro.dropna().index
    ind=factors[s].loc[ind2].dropna().index
    # Please make the fit robust 
    res=sm.OLS(factors[s].loc[ind],sm.add_constant(all_macro).loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    result_list.append(res.params)
    t_list.append(res.tvalues)

    print s
    
    print pd.DataFrame(result_list)[[ u'TED Spread',u'LIBOR term repo',
                           u'PS', u'On versus off the run Treasuries']].T.round(2).to_latex().replace('NaN',' ')
    print pd.DataFrame(t_list)[[ u'TED Spread',u'LIBOR term repo',
                           u'PS', u'On versus off the run Treasuries']].T.round(2).to_latex().replace('NaN',' ')
    

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:10: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  # Remove the CWD from sys.path while we load stuff.
/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:18: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:26: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation 

Amihud Monthly
\begin{tabular}{lrrrrrrr}
\toprule
{} &     0 &     1 &     2 &     3 &     4 &     5 &     6 \\
\midrule
TED Spread                       & -0.28 &     &  0.79 &     &     &     &  0.66 \\
LIBOR term repo                  &     &  0.58 &  0.98 &     &     &     &  0.96 \\
PS                               &     &     &     &  0.01 &     &  0.01 & -0.01 \\
On versus off the run Treasuries &     &     &     &     &  0.01 &  0.01 &  0.00 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrrrr}
\toprule
{} &     0 &     1 &     2 &     3 &     4 &     5 &     6 \\
\midrule
TED Spread                       & -0.74 &     &  1.62 &     &     &     &  1.08 \\
LIBOR term repo                  &     &  2.49 &  2.58 &     &     &     &  2.50 \\
PS                               &     &     &     &  0.36 &     &  0.48 & -0.30 \\
On versus off the run Treasuries &     &     &     &     &  0.78 &  0.83 &  0.38 \\
\bottomrule
\end{tabular}

FHT Monthly
\begin{tabular}{lrrrrrrr}
\toprule


In [14]:
print pd.DataFrame(result_list)[[ u'TED Spread',u'LIBOR term repo',
                           u'PS', u'On versus off the run Treasuries']].T.round(2).to_latex()

\begin{tabular}{lrrrrrrr}
\toprule
{} &     0 &     1 &     2 &     3 &    4 &     5 &     6 \\
\midrule
TED Spread                       &  0.17 &   NaN &  0.26 &   NaN &  NaN &   NaN &  0.25 \\
LIBOR term repo                  &   NaN & -0.02 &  0.11 &   NaN &  NaN &   NaN &  0.18 \\
PS                               &   NaN &   NaN &   NaN &  0.01 &  NaN &  0.01 & -0.04 \\
On versus off the run Treasuries &   NaN &   NaN &   NaN &   NaN & -0.0 & -0.00 & -0.01 \\
\bottomrule
\end{tabular}



In [15]:
x= pd.DataFrame(t_list)[[ u'TED Spread',u'LIBOR term repo',
                           u'PS', u'On versus off the run Treasuries']].T.round(2).to_latex()

In [16]:
x

u'\\begin{tabular}{lrrrrrrr}\n\\toprule\n{} &     0 &     1 &     2 &     3 &     4 &    5 &     6 \\\\\n\\midrule\nTED Spread                       &  0.55 &   NaN &  0.54 &   NaN &   NaN &  NaN &  0.46 \\\\\nLIBOR term repo                  &   NaN & -0.14 &  0.44 &   NaN &   NaN &  NaN &  0.72 \\\\\nPS                               &   NaN &   NaN &   NaN &  0.61 &   NaN &  0.6 & -1.28 \\\\\nOn versus off the run Treasuries &   NaN &   NaN &   NaN &   NaN & -0.09 & -0.0 & -0.46 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [17]:
results_param.append(res.params[reg_fact])
results_t.append(res.tvalues[reg_fact])

In [18]:
print res.summary()

                            OLS Regression Results                            
Dep. Variable:             FHT Annual   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                 -0.017
Method:                 Least Squares   F-statistic:                    0.5714
Date:                Thu, 06 Sep 2018   Prob (F-statistic):              0.684
Time:                        14:49:02   Log-Likelihood:                 337.12
No. Observations:                 133   AIC:                            -664.2
Df Residuals:                     128   BIC:                            -649.8
Df Model:                           4                                         
Covariance Type:                  HAC                                         
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const   